###Mount Google Drive to the Colab environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Environment Setup and Library Installation


In [ ]:
!pip install np_utils
! pip install wfdb
! pip install tensorflow
! pip install streamlit
! pip install streamlit-option-menu
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=ed24b608eed9c17b5fc7af523a2f6d734480e89f453265c34a8c1f28ce89145b
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.4/823.4 kB 12.0 MB/s eta 0:00:00


###Cloud Setup

In [ ]:
from pyngrok import ngrok
import keras.backend as K

In [ ]:

# Replace 'your_auth_token' with your actual ngrok auth token
ngrok.set_auth_token("2hWXK2xfX2Xyzbd78hlWuGMhSdO_3Wmg1JW7oC61RyrhY2ggJ")

In [ ]:
! wget -q -O - ipv4.icanhazip.com

35.185.28.111


In [ ]:
from pyngrok import ngrok

tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
    print(tunnel.public_url)

In [ ]:
# Replace 'http://<tunnel_id>.ngrok.io' with the URL of the tunnel you want to stop
ngrok.disconnect('https://d8c6-34-83-251-184.ngrok-free.app')

###Web App Using Streamlit

In [ ]:
%%writefile app.py
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, Activation, BatchNormalization, Lambda, TimeDistributed
from keras.optimizers import Adam
from tensorflow.keras.layers import Conv1D, AvgPool1D, Flatten, Dropout, Dense, Softmax
from keras.losses import categorical_crossentropy, sparse_categorical_crossentropy
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam, SGD, RMSprop
import seaborn as sns
import streamlit as st
import pywt
import matplotlib.pyplot as plt
import numpy as np
from streamlit_option_menu import option_menu
from scipy import stats
from tensorflow.keras.models import load_model
from wfdb import rdrecord,rdann,rdsamp
import os
from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import numpy as np
from tqdm import tqdm
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold
from keras.utils import to_categorical
import keras.backend as K



def zeropad(x):
    y = K.zeros_like(x)
    return K.concatenate([x, y], axis=2)

def zeropad_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3
    shape[2] *= 2
    return tuple(shape)

def ECG_model():
    """
    Implementation of the model in https://www.nature.com/articles/s41591-018-0268-3
    Also referring to codes at
    https://github.com/awni/ecg/blob/master/ecg/network.py
    and
    https://github.com/fernandoandreotti/cinc-challenge2017/blob/master/deeplearn-approach/train_model.py
    """

    model = Sequential()

    # First convolutional block
    model.add(Conv1D(filters=32, kernel_size=16, padding='same', kernel_initializer='he_normal', input_shape=(256, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=1, strides=1))

    model.add(Conv1D(filters=32, kernel_size=16, padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=32, kernel_size=16, padding='same', kernel_initializer='he_normal'))

    # Main loop blocks
    filter_length = 32
    n_blocks = 15
    for block_index in range(n_blocks):
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Conv1D(filters=filter_length, kernel_size=16, padding='same', kernel_initializer='he_normal'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
        model.add(Conv1D(filters=filter_length, kernel_size=16, padding='same', kernel_initializer='he_normal'))
        if block_index % 2 == 0:
            model.add(MaxPooling1D(pool_size=2))
        if block_index % 4 == 0 and block_index > 0:
            model.add(Lambda(zeropad, output_shape=zeropad_output_shape))
            filter_length *= 2

    # Output block
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(5, activation='softmax'))

    # adam = Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
# Compile the model


    model.summary()
    return model


K.set_image_data_format('channels_last')
x_signals1 = []
y_signals1 = []
# 1. Sidebar menu
with st.sidebar:
    selected = option_menu("NY Cardio", ["Signal Prediction", 'Model Training', 'Model Evaluation'],
                           icons=['signal', 'gear', 'check'], menu_icon="heart", default_index=0)

# Denoising function using Wavelet Transform
def denoise(data):
    w = pywt.Wavelet('sym4')
    maxlev = pywt.dwt_max_level(len(data), w.dec_len)
    threshold = 0.04  # Threshold for filtering

    coeffs = pywt.wavedec(data, 'sym4', level=maxlev)
    for i in range(1, len(coeffs)):
        coeffs[i] = pywt.threshold(coeffs[i], threshold * max(coeffs[i]))

    datarec = pywt.waverec(coeffs, 'sym4')
    return datarec
# """************************** train Model Part***************************************************"""


def save_uploaded_file(uploaded_file, upload_dir):
    file_path = os.path.join(upload_dir, uploaded_file.name)
    with open(file_path, "wb") as f:
        f.write(uploaded_file.read())
    return file_path

if selected == "Model Training":
    classes = ['N', 'F', 'Q', 'S', 'V']
    class_descriptions = {
        'N': 'Nonectopic beats',
        'F': 'Fusion beats',
        'Q': 'Unknown beats',
        'S': 'Supraventricular ectopic beats',
        'V': 'Ventricular ectopic beats'
    }
    example_beat_printed = False
    example_beat_normalized_printed = False
    UPLOAD_DIR = "uploads"
    os.makedirs(UPLOAD_DIR, exist_ok=True)

    st.write("## **Model Training**")

    st.write("#### **Train Data**")
    train_dat_file = st.file_uploader("Choose a .dat or .mat file", type=['dat', 'mat'])
    train_hea_file = st.file_uploader("Choose a .hea file", type=['hea'])
    train_atr_file = st.file_uploader("Choose a .atr file", type=['atr'])

    if train_dat_file and train_hea_file and train_atr_file:
        file_location_dat = save_uploaded_file(train_dat_file, UPLOAD_DIR)
        file_location_hea = save_uploaded_file(train_hea_file, UPLOAD_DIR)
        file_location_atr = save_uploaded_file(train_atr_file, UPLOAD_DIR)

        base_file_location_hea = file_location_hea.replace('.hea', '')
        base_file_location_atr = file_location_atr.replace('.atr', '')

        st.write("### **Preprocessing** ")
        record = rdrecord(base_file_location_hea, smooth_frames=True)
        signals1 = np.nan_to_num(record.p_signal[:, 0]).tolist()

        denoised_signals1 = denoise(signals1)
        st.write("#### **Phase 1: Data Denoising**")

        def plot_signal(signal, title, filename):
            plt.figure(figsize=(12, 6))
            plt.title(title)
            plt.plot(signal, linewidth=2)
            plt.ylabel('Amplitude')
            plt.xlabel('Sample number')
            plt.savefig(filename)
            plt.close()
            return filename

        noised_signal_plot = plot_signal(signals1[0:256*3], "Signal before denoising", "noised_signal_plot.png")
        denoised_signal_plot = plot_signal(denoised_signals1[0:256*3], "Signal after denoising", "denoised_signal_plot.png")

        denoising_expander1 = st.expander(label='Data denoising')
        with denoising_expander1:
            st.image(noised_signal_plot, caption='Noised Signal')
            st.image(denoised_signal_plot, caption='Denoised Signal')

        with st.spinner("Executing ..."):
            peaks_signals1, _ = find_peaks(denoised_signals1, distance=150)
            x_signals1, y_signals1 = [], []

            for peak in peaks_signals1[1:-1]:
                start, end = peak - 128, peak + 128
                if end > len(denoised_signals1):
                    continue
                wave = denoised_signals1[start:end]
                ann = rdann(base_file_location_atr, extension='atr', sampfrom=start, sampto=end)
                annSymbol = ann.symbol

                if not example_beat_printed:
                    segmented_heartbeat_plot = plot_signal(wave, "Segmented heartbeat", "segmented_heartbeat.png")
                    example_beat_printed = True

                wave = stats.zscore(wave)
                if not example_beat_normalized_printed:
                    normalized_heartbeat_plot = plot_signal(wave, "Heartbeat after z-score normalization", "normalized_heartbeat.png")
                    example_beat_normalized_printed = True

                if len(annSymbol) == 1 and annSymbol[0] in classes:
                    arrhythmia_index = classes.index(annSymbol[0])
                    x_signals1.append(wave)
                    y_signals1.append(arrhythmia_index)

            st.write("#### **Phase 2: Heartbeat Signal Extraction**")
            leads_expander1 = st.expander(label='Heartbeat Signal Extraction')
            with leads_expander1:
                st.image(segmented_heartbeat_plot, caption='Segmented heartbeat')

            st.write("#### **Phase 3: Normalization**")
            normalization_expander = st.expander(label='Normalization')
            with normalization_expander:
                st.image(normalized_heartbeat_plot, caption='Normalized heartbeat')

            st.write("#### **Phase 4: Mapping MIT Dataset Classes to AAMI Categories**")
            data1_expander = st.expander(label='Number of samples after the split')
            with data1_expander:
                st.write(f"All Data: {len(x_signals1)}")

        x_signals1, y_signals1 = np.array(x_signals1), np.array(y_signals1)
        train_x, val_x, train_y, val_y = train_test_split(x_signals1, y_signals1, test_size=0.20, random_state=42)
        st.write("#### **Phase 5: Split Data into Test and Validation**")

        data2_expander = st.expander(label='Number of samples after the split')
        with data2_expander:
            st.write(f"Train Data: {len(train_x)}")
            st.write(f"Validation Data: {len(val_x)}")

        st.write("#### **Phase 6: Oversampling**")
        oversampler = RandomOverSampler()
        train_x_resampled, train_y_resampled = oversampler.fit_resample(train_x, train_y)

        data3_expander = st.expander(label='Train Data after Oversampling')
        with data3_expander:
            st.write(f"Train Data after Oversampling: {len(train_x_resampled)}")

        st.write("### **Model Training** ")
        st.write("#### **Enter Training Parameters**")
        with st.form(key='training_form'):
            learning_rate = st.number_input('Learning Rate', min_value=1e-6, max_value=1.0, value=0.001, step=1e-6)
            epochs = st.number_input('Epochs', min_value=1, max_value=100, value=5, step=1)
            batch_size = st.number_input('Batch Size', min_value=1, max_value=1024, value=185, step=1)
            loss_function = st.selectbox('Loss Function', ['categorical_crossentropy', 'sparse_categorical_crossentropy'])
            optimizer = st.selectbox('Optimizer', ['adam', 'sgd', 'rmsprop'])
            submit_button = st.form_submit_button(label='Submit')

        if submit_button:
            with st.spinner("Training ..."):
                callbacks = [
                    EarlyStopping(patience=10, verbose=1),
                    ReduceLROnPlateau(factor=0.5, patience=3, min_lr=0.01, verbose=1),
                    TensorBoard(log_dir='./logs/fold_APP', histogram_freq=0, write_graph=True, write_grads=False, write_images=True),
                    ModelCheckpoint('/content/drive/MyDrive/MLII-latest_v1_fold_APP.hdf5', monitor='val_loss', save_best_only=False, verbose=1, period=10)
                ]

                # One-hot encode labels
                train_y_resampled_cat = to_categorical(train_y_resampled, num_classes=len(classes))
                val_y_cat = to_categorical(val_y, num_classes=len(classes))

                # Create and compile the model
                model = ECG_model()

                # Set optimizer
                if optimizer == 'adam':
                    opt = Adam(learning_rate=learning_rate)
                elif optimizer == 'sgd':
                    opt = SGD(learning_rate=learning_rate)
                elif optimizer == 'rmsprop':
                    opt = RMSprop(learning_rate=learning_rate)

                # Set loss function
                if loss_function == 'categorical_crossentropy':
                    loss_fn = categorical_crossentropy
                elif loss_function == 'sparse_categorical_crossentropy':
                    loss_fn = sparse_categorical_crossentropy

                model.compile(loss=loss_fn, optimizer=opt, metrics=['accuracy'])

                history = model.fit(train_x_resampled, train_y_resampled_cat,
                                    validation_data=(val_x, val_y_cat),
                                    epochs=epochs,
                                    batch_size=batch_size,
                                    callbacks=callbacks,
                                    initial_epoch=0)
                st.write("#### **Accuracy and Loss Curve**")
                train_expander = st.expander(label='Accuracy and Loss Curve')
                with train_expander:
                    def plot_training_history(history, metric, title, filename):
                        plt.figure(figsize=(12, 6))
                        plt.plot(history.history[metric], label=f'Training {metric}')
                        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
                        plt.title(title)
                        plt.xlabel('Epoch')
                        plt.ylabel(metric.capitalize())
                        plt.legend()
                        plt.savefig(filename)
                        plt.close()
                        return filename

                    loss_curve = plot_training_history(history, 'loss', 'Loss Curve', 'loss_curve.png')
                    accuracy_curve = plot_training_history(history, 'accuracy', 'Accuracy Curve', 'accuracy_curve.png')

                    st.image(loss_curve, caption='Loss Curve')
                    st.image(accuracy_curve, caption='Accuracy Curve')
                st.write("#### **Save form**")
                with st.form(key='save_form'):
                    model_name = st.text_input('Enter model name (without .h5 extension):')
                    save_button = st.form_submit_button(label='Save')

                if save_button and model_name:
                    model.save(f"{model_name}.h5")
                    st.success(f"Model saved as {model_name}.h5")
# """**************************  Evaluation Part***************************************************"""
if selected == "Model Evaluation":
    st.write("## **Model Evaluation**")
    classes = ['N', 'F', 'Q', 'S', 'V']
    class_descriptions = {
        'N': 'Nonectopic beats',
        'F': 'Fusion beats',
        'Q': 'Unknown beats',
        'S': 'Supraventricular ectopic beats',
        'V': 'Ventricular ectopic beats'
    }
    example_beat_printed = False
    example_beat_normalized_printed = False
    UPLOAD_DIR = "uploads"
    MODEL_DIR = "/content/drive/MyDrive"
    os.makedirs(UPLOAD_DIR, exist_ok=True)
    x_signals1, y_signals1 = [], []
    # Scan for .h5 files in the MODEL_DIR
    model_files = [f for f in os.listdir(MODEL_DIR) if f.endswith('.h5')]
    st.write("#### **Select Model**")
    # Model selection dropdown
    selected_model_file = st.selectbox("Choose a model for prediction", model_files)
    st.write("#### **Test Data**")
    train_dat_file = st.file_uploader("Choose a .dat or .mat file", type=['dat', 'mat'])
    train_hea_file = st.file_uploader("Choose a .hea file", type=['hea'])
    train_atr_file = st.file_uploader("Choose a .atr file", type=['atr'])

    if train_dat_file and train_hea_file and train_atr_file:
        file_location_dat = save_uploaded_file(train_dat_file, UPLOAD_DIR)
        file_location_hea = save_uploaded_file(train_hea_file, UPLOAD_DIR)
        file_location_atr = save_uploaded_file(train_atr_file, UPLOAD_DIR)

        base_file_location_hea = file_location_hea.replace('.hea', '')
        base_file_location_atr = file_location_atr.replace('.atr', '')

        st.write("### **Preprocessing** ")
        record = rdrecord(base_file_location_hea, smooth_frames=True)
        signals1 = np.nan_to_num(record.p_signal[:, 0]).tolist()

        denoised_signals1 = denoise(signals1)
        st.write("#### **Phase 1: Data Denoising**")

        def plot_signal(signal, title, filename):
            plt.figure(figsize=(12, 6))
            plt.title(title)
            plt.plot(signal, linewidth=2)
            plt.ylabel('Amplitude')
            plt.xlabel('Sample number')
            plt.savefig(filename)
            plt.close()
            return filename

        noised_signal_plot = plot_signal(signals1[0:256*3], "Signal before denoising", "noised_signal_plot.png")
        denoised_signal_plot = plot_signal(denoised_signals1[0:256*3], "Signal after denoising", "denoised_signal_plot.png")

        denoising_expander1 = st.expander(label='Data denoising')
        with denoising_expander1:
            st.image(noised_signal_plot, caption='Noised Signal')
            st.image(denoised_signal_plot, caption='Denoised Signal')

        with st.spinner("Executing ..."):
            peaks_signals1, _ = find_peaks(denoised_signals1, distance=150)


            for peak in peaks_signals1[1:-1]:
                start, end = peak - 128, peak + 128

                wave = denoised_signals1[start:end]
                ann = rdann(base_file_location_atr, extension='atr', sampfrom=start, sampto=end)
                annSymbol = ann.symbol


                if not example_beat_printed:
                    segmented_heartbeat_plot = plot_signal(wave, "Segmented heartbeat", "segmented_heartbeat.png")
                    example_beat_printed = True

                wave = stats.zscore(wave)
                if not example_beat_normalized_printed:
                    normalized_heartbeat_plot = plot_signal(wave, "Heartbeat after z-score normalization", "normalized_heartbeat.png")
                    example_beat_normalized_printed = True
                if len(annSymbol) ==1 and (annSymbol[0] in classes) :
                  if annSymbol[0] == 'F':
                    arrhythmia_index = classes.index('F')
                  elif annSymbol[0] == 'N' or annSymbol[0] == 'L' or annSymbol[0] == 'R' or annSymbol[0] == 'e' or annSymbol[0] == 'j':
                    arrhythmia_index = classes.index('N')
                  elif annSymbol[0] == '/' or annSymbol[0] == 'f' or annSymbol[0] == 'U':
                    arrhythmia_index = classes.index('Q')
                  elif annSymbol[0] == 'A' or annSymbol[0] == 'a' or annSymbol[0] == 'J' or annSymbol[0] == 'S':
                    arrhythmia_index = classes.index('S')
                  elif annSymbol[0] == 'V' or annSymbol[0] == 'E':
                    arrhythmia_index = classes.index('V')
                  x_signals1.append(wave)
                  y_signals1.append(arrhythmia_index)

            st.write("#### **Phase 2: Heartbeat Signal Extraction**")
            leads_expander1 = st.expander(label='Heartbeat Signal Extraction')
            with leads_expander1:
                st.image(segmented_heartbeat_plot, caption='Segmented heartbeat')

            st.write("#### **Phase 3: Normalization**")
            normalization_expander = st.expander(label='Normalization')
            with normalization_expander:
                st.image(normalized_heartbeat_plot, caption='Normalized heartbeat')

            st.write("#### **Phase 4: Mapping MIT Dataset Classes to AAMI Categories**")
            data1_expander = st.expander(label='Number of samples after the split')
            with data1_expander:
                st.write(f"All Data: {len(x_signals1)}")

        x_signals1 = np.array(x_signals1)
        st.write("### **Classification** ")
        # Load the model
        selected_model_path = os.path.join(MODEL_DIR, selected_model_file)
        model = load_model(selected_model_path)

        # Make predictions on the test set
        test_x = np.array(x_signals1).reshape(-1, 256, 1)
        test_predictions = model.predict(test_x)
        test_predictions = np.argmax(test_predictions, axis=1)
        predicted_labels = [classes[pred] for pred in test_predictions]

        # Get unique classes
        unique_classes = list(set(predicted_labels))
        classification_expander = st.expander(label='Classification')
        with classification_expander:
            st.write("This ECG contains the following arrhythmias:")
            for label in unique_classes:
                st.write(f"- {class_descriptions[label]} ({label})")
                    # Calculate accuracy
        accuracy = accuracy_score(y_signals1, test_predictions)


        # Calculate F1 score
        f1 = f1_score(y_signals1, test_predictions, average='weighted')
        st.write("### **accuracy and f1_score** ")
        data12_expander = st.expander(label='accuracy and f1_score ')
        with data12_expander:
          st.write(f"accuracy: {accuracy*100}%")
          st.write(f"f1_score: {f1*100}%")

        conf_matrix = confusion_matrix(y_signals1, test_predictions)
        # Calculate confusion matrix with percentages
        conf_matrix_percentage = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

        # Plot both confusion matrices
        plt.figure(figsize=(16, 8))

        # Plot confusion matrix with counts
        plt.subplot(1, 2, 1)
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
        plt.title('Confusion Matrix ')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.savefig("matrix.png")
        plt.close()

# """************************** Signal Prediction Part***************************************************"""
if selected == "Signal Prediction":
    st.write("## **Signal Prediction**")
    classes = ['N', 'F', 'Q', 'S', 'V']
    class_descriptions = {
        'N': 'Nonectopic beats',
        'F': 'Fusion beats',
        'Q': 'Unknown beats',
        'S': 'Supraventricular ectopic beats',
        'V': 'Ventricular ectopic beats'
    }
    example_beat_printed = False
    example_beat_normalized_printed = False
    UPLOAD_DIR = "uploads"
    MODEL_DIR = "/content/drive/MyDrive"
    os.makedirs(UPLOAD_DIR, exist_ok=True)
    x_signals1, y_signals1 = [], []
    # Scan for .h5 files in the MODEL_DIR
    model_files = [f for f in os.listdir(MODEL_DIR) if f.endswith('.h5')]
    st.write("#### **Select Model**")
    # Model selection dropdown
    selected_model_file = st.selectbox("Choose a model for prediction", model_files)
    st.write("#### **Test Data**")
    train_dat_file = st.file_uploader("Choose a .dat or .mat file", type=['dat', 'mat'])
    train_hea_file = st.file_uploader("Choose a .hea file", type=['hea'])
    train_atr_file = st.file_uploader("Choose a .atr file", type=['atr'])

    if train_dat_file and train_hea_file and train_atr_file:
        file_location_dat = save_uploaded_file(train_dat_file, UPLOAD_DIR)
        file_location_hea = save_uploaded_file(train_hea_file, UPLOAD_DIR)
        file_location_atr = save_uploaded_file(train_atr_file, UPLOAD_DIR)

        base_file_location_hea = file_location_hea.replace('.hea', '')
        base_file_location_atr = file_location_atr.replace('.atr', '')

        st.write("### **Preprocessing** ")
        record = rdrecord(base_file_location_hea, smooth_frames=True)
        signals1 = np.nan_to_num(record.p_signal[:, 0]).tolist()

        denoised_signals1 = denoise(signals1)
        st.write("#### **Phase 1: Data Denoising**")

        def plot_signal(signal, title, filename):
            plt.figure(figsize=(12, 6))
            plt.title(title)
            plt.plot(signal, linewidth=2)
            plt.ylabel('Amplitude')
            plt.xlabel('Sample number')
            plt.savefig(filename)
            plt.close()
            return filename

        noised_signal_plot = plot_signal(signals1[0:256*3], "Signal before denoising", "noised_signal_plot.png")
        denoised_signal_plot = plot_signal(denoised_signals1[0:256*3], "Signal after denoising", "denoised_signal_plot.png")

        denoising_expander1 = st.expander(label='Data denoising')
        with denoising_expander1:
            st.image(noised_signal_plot, caption='Noised Signal')
            st.image(denoised_signal_plot, caption='Denoised Signal')

        with st.spinner("Executing ..."):
            peaks_signals1, _ = find_peaks(denoised_signals1, distance=150)


            for peak in peaks_signals1[1:-1]:
                start, end = peak - 128, peak + 128

                wave = denoised_signals1[start:end]
                ann = rdann(base_file_location_atr, extension='atr', sampfrom=start, sampto=end)
                annSymbol = ann.symbol


                if not example_beat_printed:
                    segmented_heartbeat_plot = plot_signal(wave, "Segmented heartbeat", "segmented_heartbeat.png")
                    example_beat_printed = True

                wave = stats.zscore(wave)
                if not example_beat_normalized_printed:
                    normalized_heartbeat_plot = plot_signal(wave, "Heartbeat after z-score normalization", "normalized_heartbeat.png")
                    example_beat_normalized_printed = True
                if len(annSymbol) ==1 and (annSymbol[0] in classes) :
                  x_signals1.append(wave)
                  # y_signals1.append(arrhythmia_index)

            st.write("#### **Phase 2: Heartbeat Signal Extraction**")
            leads_expander1 = st.expander(label='Heartbeat Signal Extraction')
            with leads_expander1:
                st.image(segmented_heartbeat_plot, caption='Segmented heartbeat')

            st.write("#### **Phase 3: Normalization**")
            normalization_expander = st.expander(label='Normalization')
            with normalization_expander:
                st.image(normalized_heartbeat_plot, caption='Normalized heartbeat')

            st.write("#### **Phase 4: Mapping MIT Dataset Classes to AAMI Categories**")
            data1_expander = st.expander(label='Number of samples after the split')
            with data1_expander:
                st.write(f"All Data: {len(x_signals1)}")

        x_signals1 = np.array(x_signals1)
        st.write("### **Classification** ")
        # Load the model
        selected_model_path = os.path.join(MODEL_DIR, selected_model_file)
        model = load_model(selected_model_path)

        # Make predictions on the test set
        test_x = np.array(x_signals1).reshape(-1, 256, 1)
        test_predictions = model.predict(test_x)
        test_predictions = np.argmax(test_predictions, axis=1)
        predicted_labels = [classes[pred] for pred in test_predictions]

        # Get unique classes
        unique_classes = list(set(predicted_labels))
        classification_expander = st.expander(label='Classification')
        with classification_expander:
            st.write("This ECG contains the following arrhythmias:")
            for label in unique_classes:
                st.write(f"- {class_descriptions[label]} ({label})")






Overwriting app.py


###Start Server

In [ ]:
import os
from threading import Thread
from pyngrok import ngrok

# Function to run Streamlit
def run_streamlit():
    os.system('streamlit run /content/app.py --server.port 8501')

# Start a thread to run the Streamlit app
thread = Thread(target=run_streamlit)
thread.start()

# Open a tunnel to the Streamlit port 8501
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print('Your Streamlit app is live at:', public_url)

Your Streamlit app is live at: NgrokTunnel: "https://497b-35-185-28-111.ngrok-free.app" -> "http://localhost:8501"


###Prediction API

In [ ]:
!pip install colabcode
!pip install fastapi

In [ ]:
from colabcode import ColabCode
from fastapi import FastAPI

In [ ]:
cc.run_app(app=app)

In [ ]:

K.set_image_data_format('channels_last')
cc = ColabCode(port=12000, code=False)
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import FileResponse
from scipy import stats
from scipy.signal import find_peaks
from wfdb import rdrecord
import numpy as np
import os
from tensorflow.keras.models import load_model
import pywt
import uvicorn
import nest_asyncio
import numpy as np
from tqdm import tqdm
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn.model_selection import KFold
from keras.utils import to_categorical

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

app = FastAPI()

UPLOAD_DIR = "/content/uploads"
MODEL_PATH = "/content/drive/MyDrive/inception_model.h5"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# Load the pre-trained model
model = load_model("/content/drive/MyDrive/MLII-model_fold_3.h5")

classes = ['N', 'F', 'Q', 'S', 'V']

def denoise(data):
    w = pywt.Wavelet('sym4')
    maxlev = pywt.dwt_max_level(len(data), w.dec_len)
    threshold = 0.04  # Threshold for filtering

    coeffs = pywt.wavedec(data, 'sym4', level=maxlev)
    for i in range(1, len(coeffs)):
        coeffs[i] = pywt.threshold(coeffs[i], threshold * max(coeffs[i]))

    datarec = pywt.waverec(coeffs, 'sym4')
    return datarec

@app.post("/uploadfile/")
async def upload_file(file: UploadFile = File(...)):
    file_location = os.path.join(UPLOAD_DIR, file.filename)
    print(file.filename)
    with open(file_location, "wb") as f:
        f.write(await file.read())
    predictions = await preprocess_and_predict(file_location)
    return {"info": f"file '{file.filename}' saved and processed at '{file_location}'", "predictions": predictions}

async def preprocess_and_predict(file_path):
    try:
        num = os.path.splitext(os.path.basename(file_path))[0]
        print(num)
        record = rdrecord("/content/uploads/"+num, smooth_frames=True, channels=[0, 1])
        if record.sig_name[0] != 'MLII':
            raise ValueError("The primary signal is not 'MLII'.")

        signals0 = np.nan_to_num(record.p_signal[:, 0]).tolist()
        signals0 = denoise(signals0)

        peaks_signals0, _ = find_peaks(signals0, distance=150)

        x_signals0 = []
        for peak in peaks_signals0[1:-1]:
            start, end = peak - 128, peak + 128
            wave = signals0[start:end]

            wave = stats.zscore(wave)

            x_signals0.append(wave)

        # Convert to numpy arrays and reshape for the model
        test_x = np.array(x_signals0).reshape(-1, 256, 1)

        # Make predictions on the processed signals
        test_predictions = model.predict(test_x)
        predicted_classes = np.argmax(test_predictions, axis=1)

        # Translate predicted classes to corresponding labels
        predicted_labels = [classes[pred] for pred in predicted_classes]

        # Get unique classes
        unique_classes = list(set(predicted_labels))

        return unique_classes
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/files/{filename}")
async def get_file(filename: str):
    file_location = os.path.join(UPLOAD_DIR, filename)
    if os.path.exists(file_location):
        return FileResponse(path=file_location, filename=filename)
    return {"error": "File not found"}

# Since Colab already has an event loop, we use this method to run the app
import threading

def run_app():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    server.run()

thread = threading.Thread(target=run_app)
thread.start()
